# CIFAR-10 CNN Sınıflandırma Projesi – Genel Tanım ve Amaç

Bu not defteri, **CIFAR-10 veri kümesi üzerinde eğitilmiş bir CNN sınıflandırma modelinin**, C++ tarafında **LibTorch + OpenCV** kullanılarak nasıl modüler bir yapı ile kullanılacağını açıklamak için hazırlanmıştır.

Projenin temel amacı:

> CIFAR-10 üzerinde eğitilen bir derin öğrenme modelini TorchScript formatına dönüştürüp, C++ içinde **temiz, node tabanlı, yeniden kullanılabilir bir sınıflandırma sistemi** kurmak.

## 1. Projenin Temel Amacı

Bu proje, **tek resim sınıflandırma** görevine odaklanır.

Girdi:
- Bir görüntü dosyası (örneğin `cat.png`, `truck.jpg`, `airplane.jpg`).

Çıktı:
- CIFAR-10 sınıfları arasından tahmin edilen sınıf id'si,
- Bu sınıfa karşılık gelen sınıf adı,
- Tahminin olasılık/skor değeri.

Kısaca sistem şu soruyu yanıtlar:

> “Bu görüntü CIFAR-10 evreninde hangi sınıfa ait?”

CIFAR-10 sınıfları:

1. airplane  
2. automobile  
3. bird  
4. cat  
5. deer  
6. dog  
7. frog  
8. horse  
9. ship  
10. truck  

## 2. Yüksek Seviyede Sistem Akışı

Sınıflandırma akışı şu şekilde tasarlanmıştır:

```bash
Girdi Görsel (dosya)
        ↓
    ImageFrame
        ↓
  PreprocessNode
  (CIFAR-10 formatına uygun
   tensor üretir)
        ↓
Cifar10ClassifierNode
  (TorchScript CNN modeli ile
   forward + postprocess)
        ↓
Tahmin:
  - class_id
  - class_name
  - score
```

Bu yapı, tek bir `main.cpp` dosyasında aşağıdaki gibi kullanılabilir:

```cpp
ImageFrame frame = ...;                   // Görüntü yüklenmiş halde
torch::Tensor input = preprocess_node.process(frame);
int class_id;
std::string class_name;
float score;
classifier_node.predict(input, class_id, class_name, score);
```

Bu sayede sınıflandırma sistemi bir fonksiyon yığını değil, **iki net sorumluluğu olan node** üzerinden çalışır:

- **PreprocessNode** → Görseli modele hazır tensöre dönüştürmek,
- **Cifar10ClassifierNode** → Tensörü modele verip anlamlı tahmine çevirmek.

## 3. Kullanılan Teknolojiler

Projede temel olarak şu teknolojiler kullanılır:

- **C++17**
  - Modern C++ ile sınıf yapıları, header/implementasyon ayrımı, temiz API.

- **LibTorch (PyTorch C++ API)**  
  - TorchScript modeli yüklemek (`torch::jit::load`),
  - Tensör işlemleri (`torch::Tensor`),
  - Cihaz yönetimi (CPU / CUDA).

- **OpenCV**
  - Görüntü okuma (`cv::imread`),
  - Görüntü boyutlandırma (`cv::resize`),
  - Renk uzayı dönüşümleri (`cv::cvtColor`).

- **TorchScript**
  - Python tarafında eğitilen CIFAR-10 CNN modelinin, C++ ortamında kullanılabilmesi için TorchScript formatına dönüştürülmüş hali (`.pt` / `.ts`).

- **CMake**
  - Derleme sistemi,
  - LibTorch ve OpenCV ile bağlantı ayarları.

## 4. CIFAR-10 CNN Modeli ve TorchScript

Modelin eğitim kısmı Python tarafında gerçekleşir. Genel mantık şöyledir:

1. CIFAR-10 veri kümesi ile bir CNN modeli eğitilir.
2. Eğitim sırasında kullanılan preprocess adımları:
   - Input boyutu: 32×32,
   - Normalizasyon mean/std değerleri:
     - `mean = (0.4914, 0.4822, 0.4465)`
     - `std  = (0.2470, 0.2435, 0.2616)`
3. Eğitim tamamlandıktan sonra model TorchScript'e dönüştürülür:

```python
scripted = torch.jit.script(model)  # veya trace
scripted.save("cifar10_cnn_scripted.pt")
```

4. Elde edilen `"cifar10_cnn_scripted.pt"` dosyası C++ projesinde `models/` klasörüne konur.

C++ tarafında bu model:

- `torch::jit::load(model_path)` ile yüklenir,
- `model_.to(device)` ile CPU veya CUDA'ya taşınır,
- Inference aşamasında `model_.forward({input_tensor})` ile kullanılır.

## 5. Proje Klasör Yapısı

Önerilen proje yapısı:

```bash
Cifar10ClassifierNode/
  CMakeLists.txt
  models/
    cifar10_cnn_scripted.pt
  src/
    main.cpp
    frame_types.hpp
    cifar10_labels.hpp
    preprocess_node.hpp
    preprocess_node.cpp
    classifier_node.hpp
    classifier_node.cpp
```

### 5.1. `models/`

- `cifar10_cnn_scripted.pt`  
  TorchScript formatındaki CIFAR-10 CNN modeli burada bulunur.

### 5.2. `frame_types.hpp`

- `ImageFrame` yapısını içerir.
- Bir görüntü karesini aşağıdaki bilgilerle tutar:
  - `cv::Mat image`
  - `std::uint64_t frame_id`
  - `std::string source_name`

Bu yapı sayesinde görüntü, kimlik ve kaynak bilgisi tek bir pakette taşınır.

### 5.3. `cifar10_labels.hpp`

CIFAR-10 sınıf etiketlerini string olarak tutan basit bir header dosyasıdır. Örneğin:

```cpp
static const std::vector<std::string> CIFAR10_LABELS = {
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck"
};
```

Modelden elde edilen `class_id`, bu listede indeks olarak kullanılır ve böylece tam sınıf adı elde edilir.

### 5.4. `preprocess_node.hpp / preprocess_node.cpp`

Bu node'un görevi:

> “Her türlü giriş görüntüsünü, CIFAR-10 modelinin beklediği formatta bir `torch::Tensor`’a dönüştürmek.”

Çalışma adımları:

1. `cv::Mat` görüntüyü alır.
2. Görüntüyü `32×32` boyutuna resize eder.
3. BGR → RGB renk uzayı dönüşümünü yapar.
4. Piksel değerlerini `float` tipine çevirerek `[0, 1]` aralığına getirir.
5. `torch::from_blob` ile HWC formatındaki görüntüden bir tensor oluşturur.
6. Tensörü `[H, W, C]` → `[C, H, W]` (CHW) formatına permute eder.
7. Batch dimension eklenerek `[1, C, H, W]` formatına getirilir.
8. CIFAR-10 için kullanılan `mean` ve `std` ile kanal bazlı normalizasyon uygulanır.
9. Tensör, seçilen cihaza (`cpu` veya `cuda`) taşınır.

Bu node’un dışa açık fonksiyonu tipik olarak şöyle olur:

```cpp
torch::Tensor PreprocessNode::process(const ImageFrame& frame);
```

### 5.5. `classifier_node.hpp / classifier_node.cpp` (Cifar10ClassifierNode)

Bu node'un görevi:

> “Hazır hale getirilmiş input tensörünü TorchScript CIFAR-10 modeline verip, en olası sınıfın id’sini, adını ve skorunu üretmek.”

Temel bileşenler:

- `ClassifierConfig`:
  - `model_path`
  - `device` (`"cpu"` / `"cuda"`)
  - `class_names` (CIFAR10_LABELS)

- `Cifar10ClassifierNode`:
  - Constructor:
    - Model yoluna göre `torch::jit::load` ile modeli yükler,
    - `model_.to(device)` ile cihaz ayarını yapar,
    - Gerekirse `model_.eval()` ile inference moduna alır.
  - `predict(...)` fonksiyonu:
    - Girdi tensörünü doğru cihaza taşır,
    - `torch::NoGradGuard` ile gradient hesaplamasını kapatır,
    - `model_.forward({input})` ile forward çağrısı yapar,
    - Çıktı tensörünü `[1, num_classes]` şeklinde alır,
    - `argmax` ile en yüksek olasılıklı sınıfın index'ini bulur,
    - Gerekirse softmax hesaplayıp skor değerini elde eder,
    - `class_id`, `class_name`, `score` şeklinde dışarı verir.

### 5.6. `main.cpp`

`main.cpp`, tüm parçaları bir araya getiren basit bir çalışma senaryosu sunar.

Örnek akış:

1. Komut satırından resim yolu alınır (örn. `images/cat.png`).
2. `cv::imread` ile görüntü okunur.
3. Okunan görüntü bir `ImageFrame` içine paketlenir.
4. CIFAR-10 normalize değerleri ile bir `PreprocessConfig` oluşturulur.
5. `PreprocessNode` oluşturulur.
6. `PreprocessNode::process(frame)` çağrısıyla model giriş tensörü elde edilir.
7. `ClassifierConfig` ile `Cifar10ClassifierNode` örneği oluşturulur.
8. `predict(input_tensor, class_id, class_name, score)` çağrılır.
9. Sonuçlar konsola yazılır:

```text
Tahmin: class_id=3, class_name="cat", score=0.92
```

Bu, C++ tarafında uçtan uca çalışan bir CIFAR-10 sınıflandırma pipeline'ıdır.

## 6. Projenin Katkıları

Bu proje, aşağıdaki konularda pratik bir çerçeve sunar:

1. **Gerçek bir task’e odaklı C++ AI projesi**
   - “Bu proje ne yapıyor?” sorusuna net cevap:  
     → CIFAR-10 görsellerini sınıflandırıyor.

2. **Preprocess bilincini güçlendirme**
   - Frame → Tensor dönüşümünü dataset’e özel kurma,
   - CIFAR-10 için gerekli normalize ve boyutlandırma adımlarını doğru uygulama.

3. **TorchScript ve LibTorch kullanım pratiği**
   - Python’da eğitilmiş bir modeli C++ ortamına taşımayı öğretir.

4. **Node tabanlı mimari**
   - PreprocessNode ve Cifar10ClassifierNode ile modüler, test edilebilir bir yapı oluşturur.
   - İleride YOLO, segmentation veya başka dataset’ler için benzer node’lar tanımlamayı kolaylaştırır.

5. **Portföy değeri**
   - “CIFAR-10 CNN sınıflandırıcı node” gibi net ve anlaşılır bir proje, hem teknik mülakatlarda hem özgeçmişte gösterilebilir bir örnektir.

## 7. Özet

Bu proje:

- CIFAR-10 veri kümesi için eğitilmiş bir CNN modelini TorchScript formatında kullanarak,
- C++ tarafında LibTorch + OpenCV ile
- Temiz, modüler ve node tabanlı bir sınıflandırma pipeline’ı kurar.

Temel bileşenler:

- **ImageFrame** → Girdi görselinin taşıyıcısı,
- **PreprocessNode** → Görseli CIFAR-10 formatına uygun tensöre dönüştürür,
- **Cifar10ClassifierNode** → TorchScript modeliyle inference yapar,
- **cifar10_labels** → Sınıf id → isim dönüşümünü sağlar,
- **main.cpp** → Sistemi uçtan uca çalıştıran örnek senaryo sunar.

Bu yapı, daha karmaşık algılama ve otonom sistem projeleri için sağlam bir temel oluşturur.


-----
-----
-----
-----
-----

## Şimdi ilk yapımız ==> Python :

Öncelikle model eğitimi ve işleyişine bakalım.

* CIFAR-10 datasını otomatik indirir.


* Basit bir CNN ile eğitim yapar,

* En sonda TorchScript model (cifar10_cnn_scripted.pt) çıkarır,

* Normalizasyon değerleri C++ tarafında kullanacağımız değerlerle aynı. 


### 1️⃣ Eğitim script’i – train_cifar10.py

In [1]:
import os
import time

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import datasets, transforms


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] Using device: {device}")

[INFO] Using device: cpu


In [2]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.4914, 0.4822, 0.4465),
        std=(0.2470, 0.2435, 0.2616),
    ),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.4914, 0.4822, 0.4465),
        std=(0.2470, 0.2435, 0.2616),
    ),
])

data_root = "./data"

In [3]:

train_dataset = datasets.CIFAR10(
    root=data_root,
    train=True,
    download=True,        # DATASET BURADA OTOMATİK İNDİRİLİYOR
    transform=transform_train
)

test_dataset = datasets.CIFAR10(
    root=data_root,
    train=False,
    download=True,
    transform=transform_test
)

100%|██████████| 170M/170M [02:39<00:00, 1.07MB/s] 


In [5]:
batch_size = 128

In [6]:
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print(f"[INFO] Train set size: {len(train_dataset)}")
print(f"[INFO] Test set  size: {len(test_dataset)}")


[INFO] Train set size: 50000
[INFO] Test set  size: 10000


In [7]:
class Cifar10CNN(nn.Module):
    def __init__(self, num_classes: int = 10):
        super().__init__()

        # 3x32x32
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)   # 32x32x32
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # 64x32x32
        self.pool  = nn.MaxPool2d(2, 2)                           # → 64x16x16

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1) # 128x16x16
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)# 128x16x16
        # pool → 128x8x8

        self.fc1   = nn.Linear(128 * 8 * 8, 256)
        self.fc2   = nn.Linear(256, num_classes)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # blok 1
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)

        # blok 2
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool(x)

        # flatten
        x = torch.flatten(x, 1)

        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x


model = Cifar10CNN(num_classes=10).to(device)
print(f"[INFO] Model params: {sum(p.numel() for p in model.parameters())}")


[INFO] Model params: 2340810


In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 1


In [13]:
def train_one_epoch(epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    start_time = time.time()

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if (batch_idx + 1) % 100 == 0:
            print(f"[TRAIN] Epoch [{epoch}/{num_epochs}] "
                  f"Step [{batch_idx+1}/{len(train_loader)}] "
                  f"Loss: {loss.item():.4f}")

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = 100.0 * correct / total
    elapsed = time.time() - start_time

    print(f"[TRAIN] Epoch {epoch} finished "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.2f}%, "
          f"Time: {elapsed:.1f} sec")


In [14]:
def eval_model():
    model.eval()
    correct = 0
    total = 0
    eval_loss = 0.0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            eval_loss += loss.item() * inputs.size(0)

            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    avg_loss = eval_loss / len(test_dataset)
    acc = 100.0 * correct / total
    print(f"[EVAL] Test Loss: {avg_loss:.4f}, Acc: {acc:.2f}%")
    return avg_loss, acc

In [15]:
best_acc = 0.0
os.makedirs("checkpoints", exist_ok=True)

for epoch in range(1, num_epochs + 1):
    train_one_epoch(epoch)
    _, acc = eval_model()

    # En iyi modeli kaydet
    if acc > best_acc:
        best_acc = acc
        ckpt_path = f"checkpoints/cifar10_cnn_best.pth"
        torch.save(model.state_dict(), ckpt_path)
        print(f"[SAVE] Yeni en iyi model kaydedildi: {ckpt_path} (Acc: {best_acc:.2f}%)")

print(f"[INFO] Eğitim tamamlandı. En iyi doğruluk: {best_acc:.2f}%")

c:\Users\hdgn5\.conda\envs\torch_gpu\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


[TRAIN] Epoch [1/1] Step [100/391] Loss: 1.6283
[TRAIN] Epoch [1/1] Step [200/391] Loss: 1.4983
[TRAIN] Epoch [1/1] Step [300/391] Loss: 1.2865
[TRAIN] Epoch 1 finished Loss: 1.5249, Acc: 43.92%, Time: 124.6 sec
[EVAL] Test Loss: 1.1914, Acc: 57.12%
[SAVE] Yeni en iyi model kaydedildi: checkpoints/cifar10_cnn_best.pth (Acc: 57.12%)
[INFO] Eğitim tamamlandı. En iyi doğruluk: 57.12%


In [17]:
best_model = Cifar10CNN(num_classes=10)
best_model.load_state_dict(torch.load("checkpoints/cifar10_cnn_best.pth", map_location="cpu"))
best_model.eval()

example_input = torch.randn(1, 3, 32, 32)
scripted = torch.jit.script(best_model)
os.makedirs("models", exist_ok=True)
script_path = "models/cifar10_cnn_scripted.pt"
scripted.save(script_path)

print(f"[EXPORT] TorchScript model kaydedildi: {script_path}")

[EXPORT] TorchScript model kaydedildi: models/cifar10_cnn_scripted.pt


### **Artık python modelimiz ve eğitimimiz hazır.Artık bundan sonraki işlem C++'a kaldı.Aşağıya c++ kodlarını bırakacağım.Fakat dosyaların ana incelemesini diğer .ipynb dosyalarında yapacağız.**

----

------
----

### **Şimdi sıra C++ tarafında :**

-----
----

----

# 1️⃣ Proje klasör yapısı

```bash
Cifar10ClassifierNode/
  CMakeLists.txt
  models/
    cifar10_cnn_scripted.pt    # Python'dan export ettiğin model (buraya kopyala)
  src/
    main.cpp
    frame_types.hpp
    cifar10_labels.hpp
    preprocess_node.hpp
    preprocess_node.cpp
    classifier_node.hpp
    classifier_node.cpp
```


# 2️⃣ src/frame_types.hpp
```cpp
#ifndef FRAME_TYPES_HPP
#define FRAME_TYPES_HPP

#include <opencv2/core.hpp>
#include <cstdint>
#include <string>

// Tek bir görüntü karesini temsil eden basit yapı
struct ImageFrame
{
    cv::Mat image;              // Görüntü
    std::uint64_t frame_id{0};  // Frame numarası / kimlik
    std::string source_name;    // Kaynak (örn: "file:cat.png")

    ImageFrame() = default;

    ImageFrame(const cv::Mat& img,
               std::uint64_t id,
               const std::string& source)
        : image(img),
          frame_id(id),
          source_name(source)
    {}
};

#endif // FRAME_TYPES_HPP
```

# 3️⃣ src/cifar10_labels.hpp
```cpp
#ifndef CIFAR10_LABELS_HPP
#define CIFAR10_LABELS_HPP

#include <vector>
#include <string>

// CIFAR-10 sınıf etiketleri (sırası modelin çıktı sırasıyla aynı olmalı)
static const std::vector<std::string> CIFAR10_LABELS = {
    "airplane",   // 0
    "automobile", // 1
    "bird",       // 2
    "cat",        // 3
    "deer",       // 4
    "dog",        // 5
    "frog",       // 6
    "horse",      // 7
    "ship",       // 8
    "truck"       // 9
};

#endif // CIFAR10_LABELS_HPP
```

# 4️⃣ src/preprocess_node.hpp
```cpp
#ifndef PREPROCESS_NODE_HPP
#define PREPROCESS_NODE_HPP

#include "frame_types.hpp"

#include <torch/torch.h>
#include <string>
#include <vector>

// CIFAR-10 için preprocess ayarları
struct PreprocessConfig
{
    int input_width{32};
    int input_height{32};

    // CIFAR-10 için kullanılan normalize değerleri
    std::vector<float> mean{0.4914f, 0.4822f, 0.4465f};
    std::vector<float> std {0.2470f, 0.2435f, 0.2616f};

    std::string device{"cpu"}; // "cpu" veya "cuda"
};

// Görüntüyü CIFAR-10 modeline uygun tensöre çeviren node
class PreprocessNode
{
public:
    explicit PreprocessNode(const PreprocessConfig& config);

    // Dış API: ImageFrame -> torch::Tensor [1,3,H,W]
    torch::Tensor process(const ImageFrame& frame);

private:
    // Asıl dönüşümü yapan iç fonksiyon
    torch::Tensor preprocess(const ImageFrame& frame);

private:
    PreprocessConfig config_;
    torch::Device device_{torch::kCPU};
};

#endif // PREPROCESS_NODE_HPP
```

# 5️⃣ src/preprocess_node.cpp

```cpp
#include "preprocess_node.hpp"

#include <opencv2/imgproc.hpp>
#include <iostream>

PreprocessNode::PreprocessNode(const PreprocessConfig& config)
    : config_(config)
{
    if (config_.device == "cuda" && torch::cuda::is_available())
    {
        device_ = torch::kCUDA;
        std::cout << "[PreprocessNode] CUDA kullanılacak.\n";
    }
    else
    {
        device_ = torch::kCPU;
        std::cout << "[PreprocessNode] CPU kullanılacak.\n";
    }
}

torch::Tensor PreprocessNode::process(const ImageFrame& frame)
{
    return preprocess(frame);
}

torch::Tensor PreprocessNode::preprocess(const ImageFrame& frame)
{
    if (frame.image.empty())
    {
        std::cerr << "[PreprocessNode] HATA: Frame içindeki image boş.\n";
        return torch::empty({0});
    }

    std::cout << "[PreprocessNode] Orijinal boyut: "
              << frame.image.cols << "x" << frame.image.rows << "\n";

    // 1) Resize -> CIFAR-10 boyutu (32x32)
    cv::Mat resized;
    cv::resize(frame.image,
               resized,
               cv::Size(config_.input_width, config_.input_height));

    std::cout << "[PreprocessNode] Resize sonrası boyut: "
              << resized.cols << "x" << resized.rows << "\n";

    // 2) BGR -> RGB
    cv::Mat rgb;
    cv::cvtColor(resized, rgb, cv::COLOR_BGR2RGB);

    // 3) float'a çevir ve [0,1] aralığına getir
    cv::Mat float_img;
    rgb.convertTo(float_img, CV_32F, 1.0 / 255.0);

    const int height   = float_img.rows;
    const int width    = float_img.cols;
    const int channels = float_img.channels(); // 3

    auto options = torch::TensorOptions()
                       .dtype(torch::kFloat32)
                       .device(torch::kCPU); // ilk etapta CPU'da üretiyoruz

    // 4) cv::Mat -> torch::Tensor (HWC)
    torch::Tensor tensor = torch::from_blob(
        float_img.data,
        {height, width, channels}, // [H, W, C]
        options
    );

    // 5) HWC -> CHW
    tensor = tensor.permute({2, 0, 1}); // [C, H, W]

    // 6) Batch dimension ekle: [C, H, W] -> [1, C, H, W]
    tensor = tensor.unsqueeze(0);

    // 7) Normalize (mean / std) - CIFAR-10 değerleri
    if (config_.mean.size() == 3 && config_.std.size() == 3)
    {
        torch::Tensor mean = torch::tensor(config_.mean, options).view({1, 3, 1, 1});
        torch::Tensor std  = torch::tensor(config_.std,  options).view({1, 3, 1, 1});

        tensor = (tensor - mean) / std;
    }
    else
    {
        std::cout << "[PreprocessNode] UYARI: mean/std boyutu 3 değil, normalize atlandı.\n";
    }

    // 8) Doğru cihaza taşı
    tensor = tensor.to(device_);

    // 9) from_blob OpenCV buffer'ına bağlı olduğu için clone al
    tensor = tensor.clone();

    std::cout << "[PreprocessNode] Tensor hazır. Shape: " << tensor.sizes()
              << ", device: " << (device_.is_cuda() ? "CUDA" : "CPU") << "\n";

    return tensor;
}
```

# 6️⃣ src/classifier_node.hpp

```cpp
#ifndef CLASSIFIER_NODE_HPP
#define CLASSIFIER_NODE_HPP

#include <torch/script.h>
#include <string>
#include <vector>

// CIFAR-10 sınıflandırıcı node için config
struct ClassifierConfig
{
    std::string model_path;               // TorchScript model yolu
    std::string device{"cpu"};            // "cpu" veya "cuda"
    std::vector<std::string> class_names; // CIFAR-10 etiketleri
};

// TorchScript CIFAR-10 CNN modeliyle inference yapan node
class Cifar10ClassifierNode
{
public:
    explicit Cifar10ClassifierNode(const ClassifierConfig& config);

    // Girdi: preprocess edilmiş tensor [1,3,32,32]
    // Çıktı: class_id, class_name, score
    bool predict(const torch::Tensor& input,
                 int& out_class_id,
                 std::string& out_class_name,
                 float& out_score);

private:
    ClassifierConfig config_;
    torch::Device device_{torch::kCPU};
    torch::jit::script::Module model_;
    bool model_loaded_{false};
};

#endif // CLASSIFIER_NODE_HPP
```

# 7️⃣ src/classifier_node.cpp
```cpp
#include "classifier_node.hpp"

#include <iostream>

Cifar10ClassifierNode::Cifar10ClassifierNode(const ClassifierConfig& config)
    : config_(config)
{
    // 1) Device seç
    if (config_.device == "cuda" && torch::cuda::is_available())
    {
        device_ = torch::kCUDA;
        std::cout << "[Cifar10ClassifierNode] CUDA kullanılacak.\n";
    }
    else
    {
        device_ = torch::kCPU;
        std::cout << "[Cifar10ClassifierNode] CPU kullanılacak.\n";
    }

    // 2) Modeli yükle
    try
    {
        model_ = torch::jit::load(config_.model_path, device_);
        model_.eval();
        model_loaded_{true};
        std::cout << "[Cifar10ClassifierNode] Model yüklendi: "
                  << config_.model_path << "\n";
    }
    catch (const c10::Error& e)
    {
        std::cerr << "[Cifar10ClassifierNode] Model yüklenemedi: "
                  << e.what() << "\n";
        model_loaded_ = false;
    }
}

bool Cifar10ClassifierNode::predict(const torch::Tensor& input,
                                    int& out_class_id,
                                    std::string& out_class_name,
                                    float& out_score)
{
    if (!model_loaded_)
    {
        std::cerr << "[Cifar10ClassifierNode] HATA: Model yüklü değil.\n";
        return false;
    }

    if (!input.defined() || input.numel() == 0)
    {
        std::cerr << "[Cifar10ClassifierNode] HATA: Geçersiz input tensor.\n";
        return false;
    }

    // Girdi tensörünü doğru cihaza taşı (zaten preprocess'te taşınmış olsa bile)
    torch::Tensor tensor = input.to(device_);

    try
    {
        torch::NoGradGuard no_grad;

        // Forward çağrısı: beklenti [1, num_classes]
        std::vector<torch::jit::IValue> inputs;
        inputs.push_back(tensor);

        torch::Tensor logits = model_.forward(inputs).toTensor();

        if (logits.dim() != 2)
        {
            std::cerr << "[Cifar10ClassifierNode] Beklenen logits shape [1, num_classes], "
                      << "gelen: " << logits.sizes() << "\n";
            return false;
        }

        // Softmax ile olasılık
        torch::Tensor probs = torch::softmax(logits, 1);

        // En yüksek skor ve index
        auto max_result = probs.max(1);
        torch::Tensor max_values = std::get<0>(max_result);
        torch::Tensor max_indices = std::get<1>(max_result);

        out_class_id = max_indices.item<int>();
        out_score    = max_values.item<float>();

        if (out_class_id >= 0 &&
            out_class_id < static_cast<int>(config_.class_names.size()))
        {
            out_class_name = config_.class_names[static_cast<std::size_t>(out_class_id)];
        }
        else
        {
            out_class_name = "UNKNOWN";
        }

        return true;
    }
    catch (const c10::Error& e)
    {
        std::cerr << "[Cifar10ClassifierNode] Inference hatası: "
                  << e.what() << "\n";
        return false;
    }
}
```

# 8️⃣ src/main.cpp
```cpp
#include <iostream>
#include <opencv2/opencv.hpp>

#include "frame_types.hpp"
#include "cifar10_labels.hpp"
#include "preprocess_node.hpp"
#include "classifier_node.hpp"

int main(int argc, char** argv)
{
    // 1) Komut satırından resim yolu al
    std::string image_path = "test.png"; // varsayılan
    if (argc >= 2)
    {
        image_path = argv[1];
    }

    std::cout << "[main] Kullanılacak resim: " << image_path << "\n";

    // 2) Görüntüyü yükle
    cv::Mat img = cv::imread(image_path);
    if (img.empty())
    {
        std::cerr << "[main] HATA: Resim yüklenemedi. Yol doğru mu?\n";
        return 1;
    }

    std::cout << "[main] Resim yüklendi. Orijinal boyut: "
              << img.cols << "x" << img.rows << "\n";

    // 3) ImageFrame oluştur
    ImageFrame frame(img, 1, "file:" + image_path);

    // 4) PreprocessNode config ve oluşturma
    PreprocessConfig pre_cfg;
    pre_cfg.input_width  = 32;
    pre_cfg.input_height = 32;
    pre_cfg.mean = {0.4914f, 0.4822f, 0.4465f};
    pre_cfg.std  = {0.2470f, 0.2435f, 0.2616f};
    pre_cfg.device = "cpu"; // CUDA istersen "cuda" yap

    PreprocessNode preprocess_node(pre_cfg);

    // 5) Frame -> Tensor
    torch::Tensor input_tensor = preprocess_node.process(frame);
    if (!input_tensor.defined() || input_tensor.numel() == 0)
    {
        std::cerr << "[main] HATA: Preprocess başarısız, boş tensor.\n";
        return 1;
    }

    std::cout << "[main] Preprocess sonrası tensor shape: "
              << input_tensor.sizes() << "\n";

    // 6) Classifier config
    ClassifierConfig clf_cfg;
    clf_cfg.model_path  = "../models/cifar10_cnn_scripted.pt"; // build dizinine göre ayarla
    clf_cfg.device      = "cpu"; // CUDA istersen "cuda"
    clf_cfg.class_names = CIFAR10_LABELS;

    Cifar10ClassifierNode classifier_node(clf_cfg);

    // 7) Tahmin al
    int class_id = -1;
    std::string class_name;
    float score = 0.0f;

    bool ok = classifier_node.predict(input_tensor, class_id, class_name, score);
    if (!ok)
    {
        std::cerr << "[main] HATA: Tahmin yapılamadı.\n";
        return 1;
    }

    std::cout << "-----------------------------\n";
    std::cout << "[RESULT] class_id   = " << class_id << "\n";
    std::cout << "[RESULT] class_name = " << class_name << "\n";
    std::cout << "[RESULT] score      = " << score << "\n";
    std::cout << "-----------------------------\n";

    return 0;
}